# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 89833115,
 'calculationPrice': 'close',
 'change': 3.56,
 'changePercent': 0.0251,
 'close': 142.44,
 'closeSource': 'cfafilio',
 'closeTime': 1682311109252,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 145.32,
 'delayedPriceTime': 1728129360627,
 'extendedChange': 0.08,
 'extendedChangePercent': 0.00056,
 'extendedPrice': 148.45,
 'extendedPriceTime': 1723379541279,
 'high': 147.51,
 'highSource': 'e iemte5un dyc1ae rldip',
 'highTime': 1697995804233,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 147.363,
 'iexCloseTime': 1675152706837,
 'iexLastUpdated': 1679478063876,
 'iexMarketPercent': 0.01776067248620779,
 'iexOpen': 146.26,
 'iexOpenTime': 1689766392503,
 'iexRealtimePrice': 143.85,
 'iexRealtimeSize': 51,
 'iexVolume': 1601095,
 'lastTradeTime': 1734308800927,
 'latestPrice': 145.06,
 'latestSource': 'Close',
 'latestTime': 'June 24, 2022',
 'latestUpdate': 1705694493150,
 'latestVolume': 91198781

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [18]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [21]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,139.12,14.48,N/A
1,AOS,57.75,17.98,N/A
2,ABT,111.86,25.48,N/A
3,ABBV,152.69,22.83,N/A
4,ABMD,263.78,88.29,N/A
...,...,...,...,...
497,YUM,117.15,21.33,N/A
498,ZBRA,316.63,20.83,N/A
499,ZBH,110.82,109.04,N/A
500,ZION,53.89,5.7,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [32]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)

# get rid of negative earning stocks
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.57,2.42,N/A
1,EBAY,46.83,2.6,N/A
2,AMZN,119.77,2.8,N/A
3,SYF,30.61,4.01,N/A
4,NUE,111.51,4.01,N/A
5,F,12.43,4.21,N/A
6,MRNA,147.47,4.31,N/A
7,COF,113.65,4.32,N/A
8,AIG,54.48,4.64,N/A
9,PVH,63.92,4.74,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [33]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [35]:
portfolio_input()

That's not a number! 
 Try again:


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [38]:
position_size = float(portfolio_size) / len(final_dataframe.index) 

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.57,2.42,5185
1,EBAY,46.83,2.6,4270
2,AMZN,119.77,2.8,1669
3,SYF,30.61,4.01,6533
4,NUE,111.51,4.01,1793
5,F,12.43,4.21,16090
6,MRNA,147.47,4.31,1356
7,COF,113.65,4.32,1759
8,AIG,54.48,4.64,3671
9,PVH,63.92,4.74,3128


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [41]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [116]:
rv_columns = [
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    "Price-to-Sales Ratio",
    'PS Percetile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns 
            ),
            ignore_index=True
        )

rv_dataframe
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,140.06,N/A,14.15,N/A,5.03,N/A,2.2,N/A,9.679039,N/A,5.474365,N/A,N/A
1,AOS,58.05,N/A,17.89,N/A,4.97,N/A,2.36,N/A,11.795259,N/A,6.170983,N/A,N/A
2,ABT,114.66,N/A,26.13,N/A,5.42,N/A,4.26,N/A,17.071963,N/A,7.595388,N/A,N/A
3,ABBV,152.37,N/A,22.86,N/A,16.73,N/A,4.68,N/A,11.826583,N/A,8.213373,N/A,N/A
4,ABMD,255.71,N/A,85.68,N/A,7.9,N/A,11.19,N/A,36.000613,N/A,12.160973,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,115.28,N/A,21.08,N/A,-3.81,N/A,4.83,N/A,18.893180,N/A,8.584905,N/A,N/A
498,ZBRA,311.30,N/A,20.46,N/A,5.44,N/A,2.85,N/A,15.444252,N/A,6.457323,N/A,N/A
499,ZBH,114.14,N/A,111.6,N/A,1.91,N/A,2.93,N/A,17.832878,N/A,5.328826,N/A,N/A
500,ZION,55.92,N/A,5.64,N/A,1.37,N/A,2.3,N/A,4.299944,N/A,2.170572,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [117]:
# filtering for null and then apply to columns (axis=1 means on col)
# this will return dataframe with missing data
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
24,GOOG,2429.48,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,AON,272.14,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
64,BRK.B,288.54,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
99,CERN,95.71,N/A,48.68,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
148,DISCK,25.25,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FRT,100.67,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,FOX,30.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
331,NWS,16.39,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
362,PBCT,20.32,N/A,14.05,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
427,TROW,125.40,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [118]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)



Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [119]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [120]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    "Price-to-Sales Ratio": 'PS Percetile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
                                                                            # (entire column, value of row at col)
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore( rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,140.06,N/A,14.15,0.303785,5.03,0.699203,2.20,0.409363,9.679039,0.316733,5.474365,0.404382,N/A
1,AOS,58.05,N/A,17.89,0.416335,4.97,0.693227,2.36,0.434263,11.795259,0.422311,6.170983,0.47012,N/A
2,ABT,114.66,N/A,26.13,0.645418,5.42,0.722112,4.26,0.712151,17.071963,0.669323,7.595388,0.587649,N/A
3,ABBV,152.37,N/A,22.86,0.541833,16.73,0.930279,4.68,0.750996,11.826583,0.426295,8.213373,0.655378,N/A
4,ABMD,255.71,N/A,85.68,0.964143,7.90,0.822709,11.19,0.940239,36.000613,0.964143,12.160973,0.850598,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,115.28,N/A,21.08,0.5,-3.81,0.057769,4.83,0.76494,18.893180,0.7251,8.584905,0.679283,N/A
498,ZBRA,311.30,N/A,20.46,0.48008,5.44,0.7251,2.85,0.51494,15.444252,0.603586,6.457323,0.501992,N/A
499,ZBH,114.14,N/A,111.60,0.982072,1.91,0.284861,2.93,0.537849,17.832878,0.691235,5.328826,0.396414,N/A
500,ZION,55.92,N/A,5.64,0.069721,1.37,0.171315,2.30,0.425299,4.299944,0.067729,2.170572,0.133466,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [123]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])

    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,140.06,N/A,14.15,0.303785,5.03,0.699203,2.20,0.409363,9.679039,0.316733,5.474365,0.404382,0.426693
1,AOS,58.05,N/A,17.89,0.416335,4.97,0.693227,2.36,0.434263,11.795259,0.422311,6.170983,0.47012,0.487251
2,ABT,114.66,N/A,26.13,0.645418,5.42,0.722112,4.26,0.712151,17.071963,0.669323,7.595388,0.587649,0.667331
3,ABBV,152.37,N/A,22.86,0.541833,16.73,0.930279,4.68,0.750996,11.826583,0.426295,8.213373,0.655378,0.660956
4,ABMD,255.71,N/A,85.68,0.964143,7.90,0.822709,11.19,0.940239,36.000613,0.964143,12.160973,0.850598,0.908367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,115.28,N/A,21.08,0.5,-3.81,0.057769,4.83,0.76494,18.893180,0.7251,8.584905,0.679283,0.545418
498,ZBRA,311.30,N/A,20.46,0.48008,5.44,0.7251,2.85,0.51494,15.444252,0.603586,6.457323,0.501992,0.565139
499,ZBH,114.14,N/A,111.60,0.982072,1.91,0.284861,2.93,0.537849,17.832878,0.691235,5.328826,0.396414,0.578486
500,ZION,55.92,N/A,5.64,0.069721,1.37,0.171315,2.30,0.425299,4.299944,0.067729,2.170572,0.133466,0.173506


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [131]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

<ipython-input-131-8886e39398b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace = True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.00,14285,-3.84,0.027888,-0.9780,0.063745,0.2424,0.00996,-21.395454,0.007968,0.950823,0.035857,0.029084
1,AIG,53.67,3726,4.75,0.053785,0.7176,0.075697,0.7427,0.103586,2.588998,0.027888,0.737010,0.015936,0.055378
2,GPS,9.64,20746,-51.64,0.00996,1.3000,0.157371,0.1991,0.007968,4.800790,0.091633,0.697771,0.013944,0.056175
3,GM,36.02,5552,6.01,0.081673,0.7818,0.079681,0.3739,0.027888,1.815055,0.017928,1.516694,0.085657,0.058566
4,LNC,50.03,3997,7.31,0.103586,0.5430,0.069721,0.4190,0.033865,4.828287,0.093625,0.406644,0.001992,0.060558
5,PVH,64.33,3108,4.67,0.051793,0.8034,0.083665,0.4502,0.041833,4.607669,0.079681,1.039374,0.049801,0.061355
6,F,12.07,16570,4.35,0.047809,1.0200,0.113546,0.3530,0.025896,3.646536,0.043825,1.847495,0.101594,0.066534
7,SYF,30.00,6666,4.10,0.041833,1.1500,0.133466,0.8602,0.12749,2.501229,0.021912,0.895371,0.027888,0.070518
8,PRU,98.09,2038,8.08,0.119522,0.7995,0.081673,0.5336,0.057769,4.881180,0.09761,0.521342,0.003984,0.072112
9,MET,66.61,3002,8.59,0.133466,0.9462,0.103586,0.7123,0.093625,4.482442,0.073705,0.697721,0.011952,0.083267


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [125]:
portfolio_input()

In [126]:
portfolio_size =  float(position_size) / len(rv_dataframe.index)

for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[i, 'Price'])

rv_dataframe

/Users/kyleshum/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percetile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.00,14285,-3.84,0.027888,-0.9780,0.063745,0.2424,0.00996,-21.395454,0.007968,0.950823,0.035857,0.029084
1,AIG,53.67,3726,4.75,0.053785,0.7176,0.075697,0.7427,0.103586,2.588998,0.027888,0.737010,0.015936,0.055378
2,GPS,9.64,20746,-51.64,0.00996,1.3000,0.157371,0.1991,0.007968,4.800790,0.091633,0.697771,0.013944,0.056175
3,GM,36.02,5552,6.01,0.081673,0.7818,0.079681,0.3739,0.027888,1.815055,0.017928,1.516694,0.085657,0.058566
4,LNC,50.03,3997,7.31,0.103586,0.5430,0.069721,0.4190,0.033865,4.828287,0.093625,0.406644,0.001992,0.060558
5,PVH,64.33,3108,4.67,0.051793,0.8034,0.083665,0.4502,0.041833,4.607669,0.079681,1.039374,0.049801,0.061355
6,F,12.07,16570,4.35,0.047809,1.0200,0.113546,0.3530,0.025896,3.646536,0.043825,1.847495,0.101594,0.066534
7,SYF,30.00,6666,4.10,0.041833,1.1500,0.133466,0.8602,0.12749,2.501229,0.021912,0.895371,0.027888,0.070518
8,PRU,98.09,2038,8.08,0.119522,0.7995,0.081673,0.5336,0.057769,4.881180,0.09761,0.521342,0.003984,0.072112
9,MET,66.61,3002,8.59,0.133466,0.9462,0.103586,0.7123,0.093625,4.482442,0.073705,0.697721,0.011952,0.083267


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [132]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [133]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
    {
            'num_format': '0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
    }
)

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [134]:
column_formats = {
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percetile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBIT]DA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])

    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [135]:
writer.save()